<a href="https://colab.research.google.com/github/garkul/Transcriptomics.msu/blob/master/HW_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Домашнее задание 6</h1>
<h3> Гаркуль Лидия 4 курс ФББ</h3>

<b>Задание:</b><br>
Сделайте несколько вспомогательных функций, которые вам позднее пригодятся:
1. Разбиение одного объекта AnnData на список других, соответствующих каждому индивидуальному батчу;
2. Создание нового объекта со случайными N клетками и M генами из изначального;
3. Сохранение вашего AnnData-объекта как Seurat-объекта в формате `.rds`.

###Установка нужных программ и пакетов

In [ ]:
# Загрузка R magic
!pip install rpy2==3.5.1
%load_ext rpy2.ipython

# Установка нужных пакетов
!apt-get install libgmp3-dev libcairo2-dev libharfbuzz-dev libfribidi-dev libudunits2-dev
!gdown "1W0TuuQospxcasOX5cQdhT_Xd-k7-QAck&confirm=t"
!tar xf library.tar.gz
!pip3 install scanpy[leiden] anndata2ri

# Необходимо, чтобы конкретно прогрузился matplotlib
import os
os.kill(os.getpid(), 9)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 201 kB 5.5 MB/s 
  Created wheel for rpy2: filename=rpy2-3.5.1-cp38-cp38-linux_x86_64.whl size=310206 sha256=db53bdcf0a5dfe67f13f7c643a8425f2253aaa1d68fc48fa411997ac9f103494
  Stored in directory: /root/.cache/pip/wheels/6b/40/7d/f63e87fd83e8b99ee837c8e3489081c4b3489134bc520235ed
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.5.5
    Uninstalling rpy2-3.5.5:
      Successfully uninstalled rpy2-3.5.5
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libharfbuzz-dev is already the newest version (1.7.2-1ubuntu1).
libharfbuzz-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libcairo-script-int

In [ ]:
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython
%R .libPaths("usr/local/lib/R/site-library/")

'/content/...,'/usr/loca...,'/usr/lib/...,'/usr/lib/...


In [ ]:
!gdown "1Ctq4EmV2ps1dMiFcbqW467ywezRSGGN0&confirm=t"
!unzip PBMC_10k.zip

Downloading...
From: https://drive.google.com/uc?id=1Ctq4EmV2ps1dMiFcbqW467ywezRSGGN0&confirm=t
To: /content/PBMC_10k.zip
100% 345M/345M [00:03<00:00, 98.6MB/s]
Archive:  PBMC_10k.zip
  inflating: filtered_feature_bc_matrix.h5  
  inflating: filtered_feature_bc_matrix.tar.gz  
  inflating: metrics_summary.csv     
  inflating: raw_feature_bc_matrix.h5  
  inflating: raw_feature_bc_matrix.tar.gz  
  inflating: web_summary.html        


In [ ]:
!tar -xvzf filtered_feature_bc_matrix.tar.gz

filtered_feature_bc_matrix/
filtered_feature_bc_matrix/matrix.mtx.gz
filtered_feature_bc_matrix/features.tsv.gz
filtered_feature_bc_matrix/barcodes.tsv.gz


In [ ]:
import scanpy as sc

adata = sc.read_10x_mtx("filtered_feature_bc_matrix")

In [ ]:
import rpy2.robjects as ro

### 1. Разбиение одного объекта AnnData на список других, соответствующих каждому индивидуальному батчу

In [ ]:
#функция принимает на вход объект типа AnnData и имя колонки с batch
#функция возращает список объектов AnnData, соответствующих каждому индивидуальному батчу
def split_batch(adata, batch_col_name):
  res_list = []
  batch = adata.obs[batch_col_name].unique()
  for i in batch:
    res_list.append(adata[adata.obs[batch_col_name] == i])
  
  return res_list

In [ ]:
adata

AnnData object with n_obs × n_vars = 11996 × 36601
    var: 'gene_ids', 'feature_types'

In [ ]:
adata.obs["batch"] = [1] * 5000 + [2] * 6996
#проверка
res_list = split_batch(adata, "batch")
res_list[0].obs

,batch
AAACCCAAGGCCCAAA-1,1
AAACCCAAGTAATACG-1,1
AAACCCAAGTCACACT-1,1
AAACCCACAAAGCGTG-1,1
AAACCCACAATCGAAA-1,1
...,...
CTAGGTACAAGAGCTG-1,1
CTAGGTACAATCTGCA-1,1
CTAGGTACACTGGAAG-1,1
CTAGGTACAGGACGAT-1,1


###2. Создание нового объекта со случайными N клетками и M генами из изначального

In [ ]:
#функция принимает на вход объект типа AnnData; n - количество клеток; m - количество генов
#функция выдает объект типа AnnData со случайными N клетками и M генами из изначального
from random import sample

def random_AnnData(adata, n, m):
  res_adata = adata[sample(list(adata.obs_names), n)]
  res_adata = res_adata[:, sample(list(res_adata.var_names), m)]

  return res_adata

In [ ]:
#проверка
n = 1000
m = 2000
adata_random = random_AnnData(adata, n, m)
adata_random

View of AnnData object with n_obs × n_vars = 1000 × 2000
    obs: 'batch'
    var: 'gene_ids', 'feature_types'

###3. Сохранение вашего AnnData-объекта как Seurat-объекта в формате .rds

In [39]:
#функция принимает на вход объект типа AnnData и путь к файлу, куда сохранить 
#функция создает rds файл с AnnData как Seurat-объектом
def save_AnnData_as_rds(adata, file_name):
  ro.r("library(Seurat)")
  ro.globalenv['adata'] = adata
  ro.globalenv['file_name'] = file_name
  df = ro.r('seurat_res <- as.Seurat(adata, data=NULL, counts = "X")')
  save_rds = ro.r('saveRDS(seurat_res, file = file_name)')

In [40]:
#проверка
save_AnnData_as_rds(adata, 'adata.rds')